In [33]:
# latest preprocessing
import cv2, os, glob, h5py, yaml, math
import pathlib
import scipy.io
import tqdm
import re
import pandas as pd
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.io import savemat

In [34]:
config_path = "preprocess_config_preclinical.yaml"

with open(config_path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [35]:
def get_wmap(lmap_array):
    
    """Get weighted matrix for weighting loss for background, surfaces and boundaries
        Args:
            lmap_array (numpy array): numpy array of label
        Returns:
            wmap (numpy array): weighted matrix

    """
    
    lmap_max = np.argmax(lmap_array, axis=1) # get argmax of layers
    lmap_shifted = np.ones(lmap_max.shape) # set with background
    lmap_shifted[:,1:] = lmap_max[:,:-1] # get shifted form of layers
    lmap_diff = (lmap_max - lmap_shifted)
    lmap_diff[lmap_diff==-1] = 1
    combined = 5*(lmap_max==1)+10*lmap_diff # get weighting => last layer is backgrounds
    combined[combined==10] = 15
    wmap = combined+1
    return wmap

def convert_label_png_to_mask(annotation, HEIGHT, WIDTH):
    """Converts 2 dimentional png label input into one-hot encoded target mask
        Args:
            annotation (numpy array): numpy array of label
            HEIGHT (int): height of image 
            WIDTH (int): width of image
        Returns:
            layers (numpy array): one-hot encoded target mask

    """
    layers = np.zeros((3,HEIGHT,WIDTH)) #hardcoded for this instance, can change to max of annotation
    for h in range(HEIGHT):
        for w in range(WIDTH):
            layers[annotation[h,w], h,w] = 1
    return layers

In [36]:
class processData:
    """Class for preprocessing data 
    """
    
    def __init__(self, config):
        
        """"
        Args:
            config (dict): dictionary of configuration parameters from preprocess_config_preclinical.yaml
        """
        self.mode = config['general']['mode']
        self.sliding_window = config['general']['sliding_window']
        
        self.image_path = config['filepaths']['image_path']
        self.label_path = config['filepaths']['label_path']
        self.processed_path = config['filepaths']['processed_path']
        self.image_ext = config['filepaths']['image_ext']
        
        self.datasets = {"training":[], "val":[], "test":[]}
        self.postprocessed_data = None
        self.height = None
        self.width = None
        self.bscans = None
        self.layers = None
        self.data_list = ['data', 'lmap', 'wmap']
        
    def prepare_dataset(self, partition_idx):
        """ Obtain the folders corresponding to a specific partition idx. This is for partition training.
            Args:
                partition_idx (int): Integer corresponding to the partition idx
            Returns:
                None
        """
        idx = partition_idx
        
        #Partition splitting 
        
        self.idx = partition_idx
        dataset_splitting_train = pd.read_csv('../20220715_final_oct_synergos_split/splitting_edited_train_edited.csv')
        dataset_splitting_val = pd.read_csv('../20220715_final_oct_synergos_split/splitting_edited_val_edited.csv')
        dataset_splitting_test = pd.read_csv('../20220715_final_oct_synergos_split/splitting_edited_test_edited.csv')
        
        self.datasets['training'] = [os.path.join(*f.split(os.path.sep)[-2:]) for f in dataset_splitting_train[dataset_splitting_train['partition']==idx]['folders'].tolist()]
        
        self.datasets['val'] = [os.path.join(*f.split(os.path.sep)[-2:]) for f in dataset_splitting_val[dataset_splitting_val['partition']==idx]['folders'].tolist()]
        
        self.datasets['test'] = [os.path.join(*f.split(os.path.sep)[-2:]) for f in dataset_splitting_test[dataset_splitting_test['partition']==idx]['folders'].tolist()]

    def prepare_dataset_all(self):
        """ Obtain the folders for centralised training. 
            Args:
                None
            Returns:
                None
        """
        
        self.idx = 'all'
        dataset_splitting_train = pd.read_csv('../20220715_final_oct_synergos_split/splitting_edited_train_edited.csv')
        dataset_splitting_val = pd.read_csv('../20220715_final_oct_synergos_split/splitting_edited_val_edited.csv')
        dataset_splitting_test = pd.read_csv('../20220715_final_oct_synergos_split/splitting_edited_test_edited.csv')
        
        self.datasets['training'] = [os.path.join(*f.split(os.path.sep)[-2:]) for f in dataset_splitting_train['folders'].tolist()]
        
        self.datasets['val'] = [os.path.join(*f.split(os.path.sep)[-2:]) for f in dataset_splitting_val['folders'].tolist()]
        
        self.datasets['test'] = [os.path.join(*f.split(os.path.sep)[-2:]) for f in dataset_splitting_test['folders'].tolist()]
            
                
    def get_sliding_window(self, data_store, patient_name): # one patient
        """divide image, label, and weighted matrix into patches
            Args:
                data_store (dict): dictionary of image, label and weighted matrix array
                patient_name (str): folder name
            Returns:
                post (dict): dictionary of image, label and weighted matrix divided into patches
                all_data (list): list of dataset, scan, and the cropped location
        
        """
        
        indices = [*range(0,self.width-self.sliding_window+1,self.sliding_window)]
        post = {k:np.zeros((self.bscans, len(indices), self.height, self.sliding_window)) for k in self.data_list}
        post['lmap'] =  np.zeros((self.bscans, len(indices), self.layers, self.height,self.sliding_window))
        
        all_data = []
        for scan in range(self.bscans):
            for idx,z in enumerate(indices):
                top = 0
                bottom = self.height
                left = z
                right = z+self.sliding_window
                for datatype in self.data_list:
                    if datatype in data_store.keys():
                        if datatype == 'lmap':
                            post[datatype][scan, idx] = data_store[datatype][scan, :,top:bottom, left:right]
                        else:
                            post[datatype][scan, idx] = data_store[datatype][scan, top:bottom, left:right]
                
                all_data.append([patient_name, scan, top, bottom, left, right])

        return post, all_data
    
    def process_one_file(self, dataset):
        """read images and corresponding label for 1 folder.
            Args:
                dataset (str): file path of one folder of images 
            Returns:
                data_store (dict): dictionary of image, label and weighted matrix array
        
        """
        
        image_paths = glob.glob(os.path.join(self.image_path, dataset, '*'+self.image_ext))
        # print(f"self.image_path: {self.image_path}")
        # print(f"dataset: {dataset}")
        # print(f"image paths: {image_paths}")
        images_array = []
        lmap_array = []
    
        for i in range(len(image_paths)):
            image_path = image_paths[i]
            img = io.imread(image_path)
            if img.dtype == 'uint8':
                img = img/255
            elif img.dtype == 'uint16':
                img = img*(1.0 / 65535.0)
            
            #find corresponding label 
            image_path_stem = pathlib.Path(image_paths[i]).stem
            label_path = label_fullpath = os.path.join(self.label_path,dataset,image_path_stem+'.png')
            label = io.imread(label_path)
            lmap = convert_label_png_to_mask(label,label.shape[0], label.shape[1])
            
            
            #appending image and lmap to respective arrays
            images_array.append(img)
            lmap_array.append(lmap)

        wmap_array = get_wmap(np.array(lmap_array))
        
        return {'data':np.array(images_array), 'lmap': np.array(lmap_array), 'wmap': wmap_array}
    

     
    def create_dataset(self, mode):
        """Read and convert into patches by folder.  
            Args:
                mode (str): "training", "val" or "test"
            Returns:
                postprocessed_data (dict): stores postprocessed data for each folder.
        """

        postprocessed_data = {k:[] for k in self.data_list}
        postprocessed_data['meta_data'] = []

        for count,dataset in enumerate(tqdm.tqdm(self.datasets[mode])):

            data_store = self.process_one_file(dataset)
            self.layers = data_store['lmap'].shape[1]
            self.bscans = data_store['data'].shape[0]
            self.height = data_store['data'].shape[1]
            self.width = data_store['data'].shape[2]
            data_store, all_data = self.get_sliding_window(data_store, dataset)

            for datatype in self.data_list:
                if datatype == 'lmap':
                    postprocessed_data[datatype].append(data_store[datatype].reshape(-1, self.layers, self.height, self.sliding_window))
                else:
                    postprocessed_data[datatype].append(data_store[datatype].reshape(-1, self.height, self.sliding_window))
        
            postprocessed_data['meta_data'].append(all_data)
        for datatype in self.data_list:
                postprocessed_data[datatype] = np.concatenate(postprocessed_data[datatype], axis=0)
        return postprocessed_data
        
    def save_data(self, df, mode):
        
        """save data 
            Args:
                df (str): dataframe of patch information
                mode (str): "training", "val" or "test" 
            Returns:
                None
        """
        pathlib.Path(os.path.join(self.processed_path, 'FL_Partition_Model_' + str(self.idx))).mkdir(parents=True, exist_ok=True)
        df.to_csv('{}/{}/{}_reconstruct_data.csv'.format(self.processed_path, 'FL_Partition_Model_' + str(self.idx), mode), index=False)
        f= open("{}/{}/{}_dataset.txt".format(self.processed_path, 'FL_Partition_Model_' + str(self.idx), mode),"w+")
        for i in range(len(self.datasets[mode])):
            f.write("{}\n".format(self.datasets[mode][i]))
        f.close() 
        
        with h5py.File(os.path.join(self.processed_path, 'FL_Partition_Model_' + str(self.idx), '{}_intermediate.hdf5'.format(mode)), 'w') as hf:
            for datatype in self.data_list:
                hf.create_dataset(datatype, data=np.array(self.postprocessed_data[datatype]))
        hf.close()
        
    def make_dataset(self, mode, save):
        """Preprocess the dataset for a particular mode "training", "val" or "test".  
            Args:
                mode (str): "training", "val" or "test"
                save (bool): whether to save the data  
            Returns:
                None
        """
        if mode  == 'training' or mode == 'val':
            pathlib.Path(os.path.join(self.processed_path)).mkdir(parents=True, exist_ok=True)
            self.postprocessed_data = self.create_dataset(mode)
            # save data information and files
            flat_list = [item for sublist in self.postprocessed_data['meta_data'] for item in sublist]
            df = pd.DataFrame(flat_list, columns = ['patient_name', 'slice_number','top','bottom','left','right']) 

            if save == True: self.save_data(df, mode)
        elif mode == 'test':
            pathlib.Path(os.path.join(self.processed_path)).mkdir(parents=True, exist_ok=True)
            if save == True:
                f= open("{}/{}_dataset.txt".format(self.processed_path, mode),"w+")
                for i in range(len(self.datasets[mode])):
                    f.write("{}\n".format(self.datasets[mode][i]))
                f.close() 

In [37]:

process_data = processData(config) #may want to change selected scans
print(f"mode: {process_data.mode}")
print(f"Sliding window: {process_data.sliding_window}")
print(f"Datasets: {process_data.datasets}")      
print(f"Image path: {process_data.image_path}")
print(f"Label Path:{process_data.label_path}")
print(f"Processed path:{process_data.processed_path}")
print(f"Image Ext:{process_data.image_ext}")

mode: None
Sliding window: 64
Datasets: {'training': [], 'val': [], 'test': []}
Image path: ../20220715_final_oct_synergos/Data
Label Path:../20220715_final_oct_synergos/Segmentations
Processed path:./processed
Image Ext:.tif


In [38]:

process_data.prepare_dataset(partition_idx=1)
# print(f"Datasets: {process_data.datasets}")     
process_data.make_dataset("training", True)

if config['general']['mode'] != "all":
    process_data.make_dataset("val", True)
    process_data.make_dataset("test", True)

process_data.prepare_dataset(partition_idx=2)
process_data.make_dataset("training", True)

if config['general']['mode'] != "all":
    process_data.make_dataset("val", True)
    process_data.make_dataset("test", True)


process_data.prepare_dataset(partition_idx=3)
process_data.make_dataset("training", True)

if config['general']['mode'] != "all":
    process_data.make_dataset("val", True)
    process_data.make_dataset("test", True)
    
    


  0%|          | 0/87 [00:00<?, ?it/s]

self.image_path: ../20220715_final_oct_synergos/Data
dataset: 2018_DatasetB_Study_1_edited/4831032_1822_2907_17843_R
image paths: ['../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/4831032_1822_2907_17843_R\\NHP_0000.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/4831032_1822_2907_17843_R\\NHP_0001.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/4831032_1822_2907_17843_R\\NHP_0002.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/4831032_1822_2907_17843_R\\NHP_0005.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/4831032_1822_2907_17843_R\\NHP_0007.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/4831032_1822_2907_17843_R\\NHP_0008.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/4831032_1822_2907_17843_R\\NHP_0009.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/4831032_1822_2907_17843_R\\NHP_0010.tif

  1%|          | 1/87 [00:01<01:46,  1.24s/it]

self.image_path: ../20220715_final_oct_synergos/Data
dataset: 2018_DatasetB_Study_1_edited/13117012_1923_3307_20738_R
image paths: ['../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/13117012_1923_3307_20738_R\\NHP_0002.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/13117012_1923_3307_20738_R\\NHP_0003.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/13117012_1923_3307_20738_R\\NHP_0005.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/13117012_1923_3307_20738_R\\NHP_0006.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/13117012_1923_3307_20738_R\\NHP_0007.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/13117012_1923_3307_20738_R\\NHP_0009.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/13117012_1923_3307_20738_R\\NHP_0010.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_1_edited/13117012_1923_3307_20738_R\\NHP

  2%|▏         | 2/87 [00:03<02:38,  1.86s/it]

...

self.image_path: ../20220715_final_oct_synergos/Data
dataset: 2018_DatasetB_Study_5_edited/1627052_1926_3326_20871_R
image paths: ['../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1627052_1926_3326_20871_R\\NHP_0000.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1627052_1926_3326_20871_R\\NHP_0002.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1627052_1926_3326_20871_R\\NHP_0023.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1627052_1926_3326_20871_R\\NHP_0024.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1627052_1926_3326_20871_R\\NHP_0026.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1627052_1926_3326_20871_R\\NHP_0027.tif']


 25%|██▌       | 22/87 [00:41<01:47,  1.65s/it]

self.image_path: ../20220715_final_oct_synergos/Data
dataset: 2018_DatasetB_Study_5_edited/1691042_642_1076_7867_R
image paths: ['../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1691042_642_1076_7867_R\\NHP_0001.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1691042_642_1076_7867_R\\NHP_0002.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1691042_642_1076_7867_R\\NHP_0004.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/1691042_642_1076_7867_R\\NHP_0030.tif']


 26%|██▋       | 23/87 [00:42<01:23,  1.31s/it]

self.image_path: ../20220715_final_oct_synergos/Data
dataset: 2018_DatasetB_Study_5_edited/13117042_658_1140_8322_R
image paths: ['../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/13117042_658_1140_8322_R\\NHP_0000.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/13117042_658_1140_8322_R\\NHP_0001.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/13117042_658_1140_8322_R\\NHP_0002.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/13117042_658_1140_8322_R\\NHP_0003.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/13117042_658_1140_8322_R\\NHP_0004.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/13117042_658_1140_8322_R\\NHP_0005.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/13117042_658_1140_8322_R\\NHP_0006.tif', '../20220715_final_oct_synergos/Data\\2018_DatasetB_Study_5_edited/13117042_658_1140_8322_R\\NHP_0007.tif', '../20

In [ ]:
process_data.prepare_dataset_all()
process_data.make_dataset("training", True)

if config['general']['mode'] != "all":
    process_data.make_dataset("val", True)
    process_data.make_dataset("test", True)

100%|███████████████████████████████████████████| 89/89 [03:27<00:00,  2.34s/it]
